In [ ]:
# Ausführen um die Verbindung zur Datenbank aufzubauen

import urllib.request, os
if not os.path.isdir("resources"): os.mkdir("resources")
urllib.request.urlretrieve("https://github.com/tschuegge/SqlKurs/raw/master/resources/connect.py", "resources/connect.py")
def query(): pass
%run ./resources/connect.py

In [ ]:
# Aufgabe 1.4.1
query("""

SELECT
  ort,
  COUNT(*) anzahl
FROM kunde
GROUP BY ort

""")

In [ ]:
# Aufgabe 1.4.2
query("""

SELECT
  h.name,
  COUNT(*) anzahl
FROM artikel a
  INNER JOIN hersteller h ON a.herstellerid = h.id
GROUP BY
  a.herstellerid

""")

In [ ]:
# Aufgabe 1.4.3
query("""

SELECT
  m.name,
  m.vorname,
  SUM(a.nettopreis * p.liefermenge) nettoumsatz
FROM mitarbeiter m
  LEFT JOIN bestellungkopf k ON k.verkaeuferid = m.id
  LEFT JOIN bestellungposition p ON p.bestellungkopfid = k.id
  LEFT JOIN artikel a ON a.id = p.artikelid
GROUP BY
  m.name,
  m.vorname

-- Left Joins sind zwingend, damit alle Mitarbeitenden ausgegeben werden, auch jehne ohne Verkäufe.
-- NaN bedeutet "not a number" und ist eine Besonderheit von Sqlite. Andere DBMS würden "NULL" ausgeben.

""")

In [ ]:
# Aufgabe 1.4.4
query("""

SELECT
  m.name,
  m.vorname,
  SUM(a.nettopreis * p.liefermenge) / (m.gehalt * 12) quote
FROM mitarbeiter m
  LEFT JOIN bestellungkopf k ON k.verkaeuferid = m.id
  LEFT JOIN bestellungposition p ON p.bestellungkopfid = k.id
  LEFT JOIN artikel a ON a.id = p.artikelid
GROUP BY m.name, m.vorname  -- Da die Spalten ausgegeben werden muss nach diesen gruppiert werden
ORDER BY quote DESC

-- Viele DBMS verhalten sich nur teilweise korrekt und geben jeweils den ersten Wert aus,
-- wenn nicht gruppierte Spalten selektiert werden.

""")

In [ ]:
# Aufgabe 1.4.5
query("""

SELECT
  k.bezeichnung,
  AVG(a.nettopreis * (1 + m.satz/100)) durchschnittspreis,
  COUNT(*) anzahl
FROM artikelkategorie k
  LEFT JOIN artikel a ON a.kategorieid = k.id
  INNER JOIN mwstsatz m ON m.id = a.mwstid
WHERE k.bezeichnung <> 'Software'
GROUP BY k.bezeichnung
  HAVING anzahl > 3

""")

In [ ]:
# Aufgabe 1.4.6
query("""

SELECT
  h.name,
  SUM(a.nettopreis * p.liefermenge) lieferumsatz
FROM hersteller h
  LEFT JOIN artikel a ON a.herstellerid = h.id
  LEFT JOIN bestellungposition p ON p.artikelid = a.id
GROUP BY h.name
  HAVING lieferumsatz < 500000 OR lieferumsatz IS NULL

-- Ohne den Einschluss von Null-Werten beim Lieferumsatz, würden Hersteller ausgefiltert
-- von denen kein einziges Produkt geliefert wurde.

""")